In [1]:
import typing
import pathlib
import random
import math

In [2]:
random.seed(123456789)

In [3]:
def parse_datafile(datafile: typing.TextIO):
    """
    Returns: a pair of features and their corresponding target value
    Example:
        3 4
        0 389.05 133.34 62.14 32.60
        0 324.10 185.78 53.32 42.18
        1 438.88 161.21 88.47 51.06
    """
    header = datafile.readline().strip()
    n_instances, n_features = map(int, map(str.strip, header.split(" ")))
    lines = [datafile.readline().strip() for _ in range(n_instances)]
    X = []
    y = []
    for line in lines:
        values = [point for point in line.split(" ") if point.strip() != ""]
        y.append(int(values[0]))
        mag_X = math.sqrt(sum(math.pow(float(x), 2) for x in values[1 : n_features + 1]))
        X.append([float(x) / mag_X for x in values[1 : n_features + 1]])
    return X, y

In [4]:
datafile = pathlib.Path.cwd().joinpath('data').joinpath('L30fft16.out')
with open(datafile) as f:
    X, y = parse_datafile(f)

In [5]:
M = 9 # number of V centroids
N = len(X[0]) # number of dimensions
T = 10 # max number of time units

In [6]:
class Neuron:
    def __init__(self, xy_loc: tuple[int, int], n: int=N):
        self.weights = [random.uniform(-1.0, 1.0) for _ in range(n)]
        self.loc = xy_loc

In [7]:
def dist(V, X):
    """
    distance between two vectors in n space
    """
    return math.sqrt(sum(math.pow(v - x, 2) for v, x in zip(V, X)))

In [8]:
def distneurons(V1, V2):
    """
    distance between two neurons
    """
    return dist(V1.loc, V2.loc) # TODO: incorporate wrap-around

In [9]:
def r(t):
    return math.exp(-t/T)

In [10]:
def rbfdist(V, X, r):
    return math.exp(-r * math.pow(dist(V, X), 2))

In [11]:
def neighborhoodproximity(V1, V2, t):
    return math.exp(-t/T * math.pow(distneurons(V1, V2), 2))

In [12]:
def eta(t, initial_learning_rate):
    return initial_learning_rate * math.exp(-2 * t / T)

In [14]:
random.seed(123456789)
l_rate = 1.0
neurons = [Neuron(divmod(m, math.sqrt(M)), n=N) for m in range(M)]
before = [n.weights for n in neurons]
for t in range(T):
    x = random.choice(X)
    v0 = min(neurons, key=lambda neuron: dist(neuron.weights, x))
    # update v0 to become closer to x
    delta_v0 = eta(t, l_rate) * rbfdist(v0.weights, x, r(t))
    v0.weights = [w + delta_v0 for w in v0.weights]
    # update all vs based on neighbourhood update function
    for v in neurons:
        if v != v0:
            for j in range(N):
                v.weights[j] = v.weights[j] + eta(t, l_rate) * neighborhoodproximity(v0, v, t) * (v.weights[j] - x[j])
after = [n.weights for n in neurons]